# Debug Drill: The Wrong Clustering

**Scenario:**
A colleague segmented customers using K-means clustering.

"I found 3 distinct customer segments!" they announce proudly.

But you look at the data and something seems off. The silhouette score is poor, and the clusters don't make business sense.

**Your Task:**
1. Identify TWO bugs in the clustering pipeline
2. Fix the issues
3. Write a 3-bullet postmortem

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.datasets import make_moons

np.random.seed(42)

In [ ]:
# Generate synthetic customer data with non-spherical structure
# This represents customers who cluster along behavioral patterns

# Create two crescent-shaped groups (like customers with different engagement curves)
X_moons, y_true = make_moons(n_samples=400, noise=0.08, random_state=42)

# Scale to realistic customer metrics
# Feature 1: Monthly spend ($10-$500 range)
# Feature 2: Engagement score (0-100 range)
X_customers = X_moons.copy()
X_customers[:, 0] = X_moons[:, 0] * 150 + 200  # Spend: ~50-350
X_customers[:, 1] = X_moons[:, 1] * 30 + 50     # Engagement: ~20-80

df = pd.DataFrame(X_customers, columns=['monthly_spend', 'engagement_score'])
df['true_segment'] = y_true  # Ground truth (unknown to algorithm)

print("Customer Data Statistics:")
print(df.describe().round(1))

In [ ]:
# ===== COLLEAGUE'S CODE (TWO BUGS!) =====

# BUG 1: No feature scaling (spend and engagement are on different scales)
X = df[['monthly_spend', 'engagement_score']].values  # <-- NOT SCALED!

# BUG 2: Using K-means on non-spherical data (should use DBSCAN)
kmeans = KMeans(n_clusters=3, random_state=42)  # <-- Wrong K and wrong algorithm!
df['cluster'] = kmeans.fit_predict(X)

sil_score = silhouette_score(X, df['cluster'])

print("=== Colleague's Results ===")
print(f"Number of clusters: 3")
print(f"Silhouette Score: {sil_score:.3f}")
print("\n🤔 Silhouette is low... is the clustering actually good?")

In [ ]:
# Visualize the problem
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# True clusters
axes[0].scatter(df['monthly_spend'], df['engagement_score'], c=df['true_segment'], cmap='viridis', alpha=0.6)
axes[0].set_xlabel('Monthly Spend ($)')
axes[0].set_ylabel('Engagement Score')
axes[0].set_title('TRUE Segments (2 groups)')

# K-means clusters (wrong)
scatter = axes[1].scatter(df['monthly_spend'], df['engagement_score'], c=df['cluster'], cmap='tab10', alpha=0.6)
# Add centroids
centers = kmeans.cluster_centers_
axes[1].scatter(centers[:, 0], centers[:, 1], c='red', marker='X', s=200, label='Centroids')
axes[1].set_xlabel('Monthly Spend ($)')
axes[1].set_ylabel('Engagement Score')
axes[1].set_title(f'K-Means (K=3, unscaled) - Silhouette: {sil_score:.2f}')
axes[1].legend()

# Placeholder for fixed version
axes[2].scatter(df['monthly_spend'], df['engagement_score'], c='gray', alpha=0.3)
axes[2].set_xlabel('Monthly Spend ($)')
axes[2].set_ylabel('Engagement Score')
axes[2].set_title('TODO: Fix the clustering')

plt.tight_layout()
plt.show()

print("\n❌ K-means cuts through the crescents instead of separating them!")
print("   The algorithm assumes spherical clusters, but these are moon-shaped.")

---

## Your Investigation

### Step 1: Identify the bugs

In [ ]:
print("=== Bug Analysis ===")
print()
print("❌ BUG 1: No feature scaling")
print(f"   Monthly spend range: {df['monthly_spend'].min():.0f} - {df['monthly_spend'].max():.0f}")
print(f"   Engagement range:    {df['engagement_score'].min():.0f} - {df['engagement_score'].max():.0f}")
print("   → Spend has 3-4x larger range, dominating distance calculations")
print()
print("❌ BUG 2: K-means on non-spherical data")
print("   K-means assumes clusters are spherical (blob-shaped)")
print("   These customer segments are crescent-shaped")
print("   → Should use DBSCAN or adjust the data")

### Step 2: TODO - Fix the clustering

In [ ]:
# TODO: Fix the clustering issues

# Uncomment and complete:

# # Fix 1: Scale the features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(df[['monthly_spend', 'engagement_score']])
# 
# # Fix 2: Use DBSCAN for non-spherical clusters
# dbscan = DBSCAN(eps=0.3, min_samples=10)
# df['cluster_fixed'] = dbscan.fit_predict(X_scaled)
# 
# # Evaluate
# n_clusters = len(set(df['cluster_fixed'])) - (1 if -1 in df['cluster_fixed'].values else 0)
# n_noise = (df['cluster_fixed'] == -1).sum()
# 
# print("=== Fixed Clustering Results ===")
# print(f"Clusters found: {n_clusters}")
# print(f"Noise points: {n_noise}")
# 
# if n_clusters >= 2:
#     # Calculate silhouette excluding noise
#     mask = df['cluster_fixed'] != -1
#     sil_fixed = silhouette_score(X_scaled[mask], df.loc[mask, 'cluster_fixed'])
#     print(f"Silhouette Score: {sil_fixed:.3f}")

In [ ]:
# TODO: Compare results

# Uncomment:

# # Visualize fixed clustering
# fig, axes = plt.subplots(1, 2, figsize=(12, 4))
# 
# # True clusters
# axes[0].scatter(df['monthly_spend'], df['engagement_score'], c=df['true_segment'], cmap='viridis', alpha=0.6)
# axes[0].set_title('True Segments')
# axes[0].set_xlabel('Monthly Spend ($)')
# axes[0].set_ylabel('Engagement Score')
# 
# # DBSCAN clusters
# axes[1].scatter(df['monthly_spend'], df['engagement_score'], c=df['cluster_fixed'], cmap='viridis', alpha=0.6)
# axes[1].set_title(f'DBSCAN (scaled) - {n_clusters} clusters found')
# axes[1].set_xlabel('Monthly Spend ($)')
# axes[1].set_ylabel('Engagement Score')
# 
# plt.tight_layout()
# plt.show()

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

# Uncomment:

# assert 'cluster_fixed' in df.columns, "Should have created fixed clusters"
# n_found = len(set(df['cluster_fixed'])) - (1 if -1 in df['cluster_fixed'].values else 0)
# assert n_found == 2, f"Should find 2 clusters, found {n_found}"
# 
# print("✓ Clustering fixed!")
# print(f"✓ Found {n_found} clusters (matching true segments)")
# print(f"✓ Silhouette improved: {sil_score:.3f} → {sil_fixed:.3f}")

### Step 3: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: The Wrong Clustering

### What happened:
- (Your answer: What were the symptoms that indicated bad clustering?)

### Root cause:
- (Your answer: What two bugs caused the poor clustering?)

### How to prevent:
- (Your answer: What should we always check before running K-means?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lessons:**

1. **Always scale features** before K-means — it uses Euclidean distance.

2. **K-means assumes spherical clusters.** Use DBSCAN for arbitrary shapes.

3. **Low silhouette score is a red flag.** Investigate before proceeding.

4. **Visualize your clusters.** Metrics alone don't tell the whole story.

---

## Algorithm Selection Guide

| Data Shape | Algorithm | Why |
|------------|-----------|-----|
| Spherical blobs | K-Means | Fast, simple, works well |
| Arbitrary shapes | DBSCAN | Finds density-based clusters |
| Hierarchical | Agglomerative | Creates dendrograms |
| Unknown K | DBSCAN or Hierarchical | Don't need to specify K |